<a target="_blank" href="https://colab.research.google.com/github/jackma-00/house-price-prediction/blob/main/experiments/1_house_price_feature_backfill.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# <span style="font-width:bold; font-size: 3rem; color:#333;">Feature Backfill<span>

Installing required packages

In [1]:
!pip install hopsworks[python]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 7.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.6/258.6 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 66.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 640.3/640.3 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Imports

In [2]:
import pandas as pd
import hopsworks
import os

import warnings
warnings.filterwarnings("ignore")

### <span style='color:#ff5f27'> Connect to Hopsworks Feature Store

In [4]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
os.environ["HOPSWORKS_API_KEY"] = ""
proj = hopsworks.login()


Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1158295


## <span style="color:#ff5f27;"> 💽 Loading the Data </span>

In [5]:
# Connect to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# unzip properties.zip
!unzip /content/drive/MyDrive/properties.zip -d /content/

Archive:  /content/drive/MyDrive/properties.zip
  inflating: /content/properties.csv  


In [9]:
# unzip omi.zip
!unzip /content/drive/MyDrive/omi.zip -d /content/

Archive:  /content/drive/MyDrive/omi.zip
  inflating: /content/omi.csv        


In [7]:
# Load properties CSV into Dataframe
properties_csv = 'properties.csv'
properties_df = pd.read_csv(properties_csv)

In [10]:
# Load omi CSV into Dataframe
omi_csv = 'omi.csv'
omi_df = pd.read_csv(omi_csv)

In [ ]:
properties_df

In [ ]:
omi_df

## <span style="color:#ff5f27;"> Data Cleaning

In [ ]:
properties_df.info()

In [ ]:
omi_df.info()

Drop null values

In [15]:
properties_df.dropna(inplace=True)

In [16]:
omi_df.dropna(inplace=True)

Convert column names to lowercase

In [17]:
# Convert column names to lowercase
properties_df.columns = properties_df.columns.str.lower()

In [18]:
# Convert column names to lowercase
omi_df.columns = omi_df.columns.str.lower()

**Evaluating candidate features for properties fg**

Primary key:
* id

Timestamp:
* scraping_date

Numerical features:
* agencyid
* bedroomsnumber
* buildingyear
* codcom
* gsm
* surface
* latitude
* longitude

Boolean features:
* isluxury
* isnew
* on_the_market
* zeroenergybuilding

Categorical features:
* airconditioning
* bathrooms
* city
* condition
* energyclass
* ga4heating
* garage
* heatingtype
* pricerange
* rooms
* id_zona_omi

Label:
* price

In [19]:
# properties selected columns
properties_df = properties_df[[
    'id',
    'scraping_date',
    'agencyid',
    'bedroomsnumber',
    'buildingyear',
    'codcom',
    'gsm',
    'surface',
    'latitude',
    'longitude',
    'isluxury',
    'isnew',
    'on_the_market',
    'zeroenergybuilding',
    'airconditioning',
    'bathrooms',
    'city',
    'condition',
    'energyclass',
    'ga4heating',
    'garage',
    'heatingtype',
    'pricerange',
    'rooms',
    'id_zona_omi',
    'price',
]]

In [32]:
properties_df['scraping_date'] = pd.to_datetime(properties_df['scraping_date'])
properties_df = properties_df.sort_values(by=['scraping_date'])

In [33]:
properties_df

,id,scraping_date,agencyid,bedroomsnumber,buildingyear,codcom,gsm,surface,latitude,longitude,...,city,condition,energyclass,ga4heating,garage,heatingtype,pricerange,rooms,id_zona_omi,price
669203,103007050,2023-09-26,139450.0,3.0,1970.0,99013.0,140.0,110,44.0007,12.6411,...,Riccione,Buono / Abitabile,G,Autonomo,1 in box privato/box in garage,"autonomo, a radiatori, alimentato a metano",200.001 - 300.000 &euro;,4,H274-D2,275000.0
668940,102157942,2023-09-26,381791.0,2.0,1980.0,39014.0,179.0,110,44.3758,12.2386,...,Ravenna,Buono / Abitabile,E,Autonomo,"1 in box privato/box in garage, 2 in parcheggi...","autonomo, a radiatori, alimentato a metano",200.001 - 300.000 &euro;,3,H199-D1,220000.0
668907,105450397,2023-09-26,105721.0,1.0,2000.0,38008.0,31.0,60,44.8215,11.6198,...,Ferrara,Ottimo / Ristrutturato,D,Autonomo,1 in box privato/box in garage,"autonomo, a radiatori, alimentato a metano",100.001 - 150.000 &euro;,2,D548-C3,135000.0
668752,92487240,2023-09-26,164824.0,1.0,1951.0,88009.0,657.0,75,36.9218,14.7308,...,Ragusa,Buono / Abitabile,G,Autonomo,2 in parcheggio/garage comune,"autonomo, ad aria, alimentazione elettrica",fino a 100.000 &euro;,2,H163-B1,35000.0
667832,98273200,2023-09-26,182966.0,2.0,1980.0,82053.0,376.0,120,38.0971,13.3688,...,Palermo,Buono / Abitabile,F,Autonomo,1 in parcheggio/garage comune,"autonomo, a radiatori, alimentato a metano",fino a 100.000 &euro;,4,G273-D3,78000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451987,105046285,2024-04-01,5689.0,3.0,2023.0,13075.0,249.0,149,45.7993,9.0906,...,Como,Nuovo / In costruzione,A2,Autonomo,1 in box privato/box in garage,"autonomo, a pavimento, alimentato a pompa di c...",oltre 500.000 &euro;,4,C933-C1,505000.0
444959,105308075,2024-04-01,224012.0,3.0,2002.0,89017.0,233.0,390,37.0808,15.2732,...,Siracusa,Ottimo / Ristrutturato,F,Autonomo,1 in box privato/box in garage,"autonomo, ad aria, alimentato a metano",oltre 500.000 &euro;,5,I754-C2,520000.0
447661,104521685,2024-04-01,388920.0,2.0,2023.0,48017.0,269.0,85,43.7752,11.3011,...,Firenze,Nuovo / In costruzione,A2,Centralizzato,"1 in box privato/box in garage, 1 in parcheggi...","centralizzato, ad aria, alimentato a metano",oltre 500.000 &euro;,4,D612-D11,550000.0
448706,103633674,2024-04-01,74482.0,2.0,2023.0,42021.0,303.0,101,43.5276,13.2150,...,Jesi,Nuovo / In costruzione,A4,Autonomo,1 in box privato/box in garage,"autonomo, a pavimento",300.001 - 500.000 &euro;,3,E388-D3,327000.0


In [22]:
omi_df = omi_df[[
    'id_zona_omi',
    'tipologia',
    'condizione',
    'compr_min',
    'compr_max',
    'zona_omi_type',
]]

In [23]:
omi_df

,id_zona_omi,tipologia,condizione,compr_min,compr_max,zona_omi_type
0,A048-B1,Abitazioni civili,normale,812.0,1165.0,B
1,A048-B1,Abitazioni civili,ottimo,1062.0,1454.0,B
2,A048-B1,Abitazioni civili,scadente,539.0,787.0,B
3,A048-B1,Abitazioni di tipo economico,normale,650.0,930.0,B
4,A048-B1,Abitazioni di tipo economico,ottimo,892.0,1192.0,B
...,...,...,...,...,...,...
245119,M422-E1,Abitazioni di tipo economico,ottimo,787.0,918.0,E
245120,M422-E1,Abitazioni di tipo economico,scadente,328.0,412.0,E
245121,M422-E1,Ville e Villini,normale,680.0,832.0,E
245122,M422-E1,Ville e Villini,ottimo,920.0,1050.0,E


## <span style="color:#ff5f27;"> 🪄 Creating Feature Groups </span>

In [34]:
fs = proj.get_feature_store()

In [35]:
# Get or create the 'properties' feature group
properties_fg = fs.get_or_create_feature_group(
    name='properties',
    version=3,                                                 # Breaking version
    description='Property Features and Corresponding Prices',
    online_enabled=True,                                       # Necessary for online inference
    primary_key=['id'],
    event_time='scraping_date',
)

In [36]:
# Insert data into feature group
properties_fg.insert(properties_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1158295/fs/1148998/fg/1392971


Uploading Dataframe: 100.00% |██████████| Rows 11232/11232 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: properties_3_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1158295/jobs/named/properties_3_offline_fg_materialization/executions


(Job('properties_3_offline_fg_materialization', 'SPARK'), None)

In [37]:
omi_fg = fs.get_or_create_feature_group(
    name='omi',
    version=2,
    description='OMI information',
    online_enabled=True,
    primary_key=['id_zona_omi'],
)

In [38]:
# Insert data into feature group
omi_fg.insert(omi_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1158295/fs/1148998/fg/1392972


Uploading Dataframe: 100.00% |██████████| Rows 240570/240570 | Elapsed Time: 00:14 | Remaining Time: 00:00


Launching job: omi_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1158295/jobs/named/omi_2_offline_fg_materialization/executions


(Job('omi_2_offline_fg_materialization', 'SPARK'), None)